---
title: "뉴스 기사의 감정분석하기"
author: "양성모"
date: "8/25/2023"
categories: [HuggingFace]
---

경제 기사의 감정을 평가하는 모델을 사용하는 방법을 정리합니다.

# 시작하기 전에

프로젝트는 transformers(ver. 4.28.0)와 tokenizer(ver. 0.13.3)를 사용했습니다. 문제가 발생하는 경우 버전을 확인하세요.


# 뉴스 기사 감정 분석하기


허깅페이스에는 테스크 목적과 모델의 언어에 따라 분류된 미세 조정 모델을 저장하고 있습니다. 이 모델을 이용하여 뉴스의 감정을 평가하는 모델을 사용하는 프로젝트를 연습합니다. transformer를 사용하기 위해서 transformer의 pipeline과 토크나이저를 가져옵니다.

In [ ]:
!pip install transformers==4.28.0
!pip install tokenizers==0.13.3
!pip install sentensepiece
from transformers import AutoTokenizer, pipeline

토크나이저는 텍스트를 모델이 처리할 수 있는 데이터 형태로 변경합니다. 문자 형태의 입력은 토크나이저를 통해서 모델이 이해할 수 있는 숫자 형태로 변경되고 항상  모델에 맞는 토크나이저를 사용해야 합니다.

토크나이저를 테스트 하기 위해 `xlm-reberta` 모델을 한국어 뉴스 데이터 셋으로 미세 조정한 체크포인트를 사용하겠습니다. 현서님이 작업한 체크포인트를 시작으로 다시 미세 조정을 수행했습니다.



## 토크나이저

`AutoTokenizer`를 이용하면 트랜스포머 모델의 이름을 입력으로 사용하여 맞는 토크나이저를 선택할 수 있습니다. 아래와 같이 체크포인트 이름으로 모델에 맞는 토크나이저를 선택해보겠습니다

In [6]:
model_name = 'gabrielyang/finance_news_classifier-KR_v7'
tokenizer = AutoTokenizer.from_pretrained(model_name)

만약 토크나이저 실행 시 `protobuf` 에러가 발생한다면 `pip install protobuf==3.20` 명령으로  3.20 버전으로 설치해보세요.

::: {.callout-note}

sentencepiece는 왜 설치가 필요할까요? 
[SentencePiece](https://github.com/google/sentencepiece)는 Google에서 2018년도에 공개한 오픈소스 라이브러리로, 단어를 구분하기 위해 공백을 사용한다는 가정으로 발생하는 문제를 해결하며 다양한 자연어처리 태스크에서 널리 사용되고 있습니다. 

:::

토크나이저를 이용해서 문자열이 어떻게 변경되는 지 확인합니다. 문자열을 토크나이저의 입력으로 전달합니다.

In [7]:
input_text = """급증하는 가계부채 부담에도 경기회복이 지연될 것이란 우려에 기준금리는 5회 연속 동결했다."""
len(input_text)

50

len명령으로 글자가 50자임을 확인 했습니다. 토크나이저를 사용한 후 길이를 확인해보겠습니다.

In [8]:
input_ids = tokenizer(input_text)
display(input_ids)
display(len(input_ids['input_ids']))

{'input_ids': [0, 67448, 18128, 2123, 6081, 11931, 3614, 44115, 105494, 20584, 45376, 6192, 23224, 469, 6339, 7956, 24788, 10454, 15916, 165034, 480, 74108, 189792, 769, 190, 6192, 150813, 8354, 25354, 5476, 5, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

32

출력은 'input_ids' 키를 갖는 숫자 리스트 형태로 리턴었고 길이는 32로 줄었습니다. 입력 ID(input ids)는 모델의 입력으로 전달되는 숫자 표현의 토큰 인덱스입니다.

## Pipeline

파이프라인은 데이터를 전처리 및 변환하고 모델에서 예측을 반환하기 위해 특정 순서로 실행되는 일련의 단계를 나타내는 추상화입니다. 파이프라인에서 발견되는 몇 가지 예시 단계는 데이터 전처리, 특징 추출 및 정규화일 수 있습니다.

문자열 입력을 모델에 사용하기 위해 필요한 토크나이저도 파이프라인에 포함되어 있기 때문에 아래의 코드에서는 문자열 입력을 추론으로 사용합니다.

In [11]:
model_name = 'gabrielyang/finance_news_classifier-KR_v7'
classifier = pipeline(model=model_name)

In [13]:
input_text

'급증하는 가계부채 부담에도 경기회복이 지연될 것이란 우려에 기준금리는 5회 연속 동결했다.'

In [12]:
classifier(input_text)

[{'label': 'negative', 'score': 0.9994288086891174}]

추론 결과는 딕셔너리 형태로 `label`정보와 `score`로 리턴됩니다. 입력 문자열은 `부정적`으로 판단되었습니다.